In [1]:
import math
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import lxml
import re
from random import seed
from collections import Counter
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import array
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import desc

C:\Users\MoJo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
LinkIndoor400mMen = "http://www.ustfccca.org/records-lists/outdoor-collegiate-all-time-bests?sport=1&top=25&event=4"

In [3]:
page = requests.get(LinkIndoor400mMen)
soup = BeautifulSoup(page.text,'html')
soup

table = soup.find("table")

In [4]:
Header = soup.findAll('th')
Headers = []
for h in Header:
    Headers.append(h.text)

In [5]:
Headers

['Athlete', 'Institution', 'Mark', 'Date', '']

In [6]:
row_soup = soup.findAll('tr')

In [7]:
row_soup

[<tr><th style="width: 35%;">Athlete</th><th colspan="2" style="width: 30%;">Institution</th><th style="width: 25%;">Mark</th><th style="width: 10%;">Date</th><th></th></tr>,
 <tr><td colspan="100" style="font-weight: bolder;">UNOFFICIAL Top 25 Performers and Performances</td></tr>,
 <tr><td colspan="5"><div class="container-table" style="width: 100%;"><table class="embed" width="100%"><thead><tr></tr></thead><tr><td style="width: 35%;"><span><a href="https://tfrrs.org/athletes/6110322" target="_blank">(1) Michael NORMAN</a></span><span style="float:right;font-size:0.9em;">SO</span></td><td style="width: 5%;"><span class="group-logo-inline"><img alt="Southern California" class="group-logo-inline" src="/images/logos/rankings-article/southern-california.png" title="Southern California"/></span></td><td style="width: 25%;"><a href="/records-lists/outdoor-collegiate-all-time-bests?sport=1&amp;top=25&amp;event=4&amp;team=1432">Southern California</a></td><td style="width: 25%;"><a href="htt

In [8]:
rows = []
for row in row_soup:
    row_data = []
    for cell in row.findAll('td'):
        row_data.append(cell.text)
    rows.append(row_data)

In [9]:
rows

[[],
 ['UNOFFICIAL Top 25 Performers and Performances'],
 ['(1) Michael NORMANSOSouthern California44.52📺 3/10/2018\xa0\xa0\xa0\xa0\xa021.33, 23.20',
  '(1) Michael NORMANSO',
  '',
  'Southern California',
  '44.52📺 ',
  '3/10/2018',
  '\xa0\xa0\xa0\xa0\xa021.33, 23.20'],
 [],
 ['(1) Michael NORMANSO', '', 'Southern California', '44.52📺 ', '3/10/2018'],
 ['\xa0\xa0\xa0\xa0\xa021.33, 23.20'],
 ['(2) Kerron CLEMENTSOFlorida44.57📺 3/12/2005',
  '(2) Kerron CLEMENTSO',
  '',
  'Florida',
  '44.57📺 ',
  '3/12/2005'],
 [],
 ['(2) Kerron CLEMENTSO', '', 'Florida', '44.57📺 ', '3/12/2005'],
 ['(3) Randolph ROSSJRNorth Carolina A&T44.62 3/12/2022',
  '(3) Randolph ROSSJR',
  '',
  'North Carolina A&T',
  '44.62 ',
  '3/12/2022'],
 [],
 ['(3) Randolph ROSSJR', '', 'North Carolina A&T', '44.62 ', '3/12/2022'],
 ['(4) Noah WILLIAMSJRLSU44.71 3/13/2021',
  '(4) Noah WILLIAMSJR',
  '',
  'LSU',
  '44.71 ',
  '3/13/2021'],
 [],
 ['(4) Noah WILLIAMSJR', '', 'LSU', '44.71 ', '3/13/2021'],
 ['(5) Elija 

In [10]:
len(rows)

173

In [11]:
rows[124]

['\xa0\xa0\xa0\xa0— Bloomfield {3}SO',
 '',
 'Auburn',
 '45.22 bb = Performance occurred on a 200m/220yd banked track ',
 '2/25/2018']

In [12]:
len(rows)

173

In [13]:
RealRows = []

In [14]:
for i in range(0,124):
    if len(rows[i]) > 1:
        RealRows.append(rows[i])
        

In [15]:
RealRows

[['(1) Michael NORMANSOSouthern California44.52📺 3/10/2018\xa0\xa0\xa0\xa0\xa021.33, 23.20',
  '(1) Michael NORMANSO',
  '',
  'Southern California',
  '44.52📺 ',
  '3/10/2018',
  '\xa0\xa0\xa0\xa0\xa021.33, 23.20'],
 ['(1) Michael NORMANSO', '', 'Southern California', '44.52📺 ', '3/10/2018'],
 ['(2) Kerron CLEMENTSOFlorida44.57📺 3/12/2005',
  '(2) Kerron CLEMENTSO',
  '',
  'Florida',
  '44.57📺 ',
  '3/12/2005'],
 ['(2) Kerron CLEMENTSO', '', 'Florida', '44.57📺 ', '3/12/2005'],
 ['(3) Randolph ROSSJRNorth Carolina A&T44.62 3/12/2022',
  '(3) Randolph ROSSJR',
  '',
  'North Carolina A&T',
  '44.62 ',
  '3/12/2022'],
 ['(3) Randolph ROSSJR', '', 'North Carolina A&T', '44.62 ', '3/12/2022'],
 ['(4) Noah WILLIAMSJRLSU44.71 3/13/2021',
  '(4) Noah WILLIAMSJR',
  '',
  'LSU',
  '44.71 ',
  '3/13/2021'],
 ['(4) Noah WILLIAMSJR', '', 'LSU', '44.71 ', '3/13/2021'],
 ['(5) Elija GODWINSRGeorgia44.75📺 3/11/2023',
  '(5) Elija GODWINSR',
  '',
  'Georgia',
  '44.75📺 ',
  '3/11/2023'],
 ['(5) Eli

In [16]:
RealRealRows = RealRows[::2]
RealRealRows

[['(1) Michael NORMANSOSouthern California44.52📺 3/10/2018\xa0\xa0\xa0\xa0\xa021.33, 23.20',
  '(1) Michael NORMANSO',
  '',
  'Southern California',
  '44.52📺 ',
  '3/10/2018',
  '\xa0\xa0\xa0\xa0\xa021.33, 23.20'],
 ['(2) Kerron CLEMENTSOFlorida44.57📺 3/12/2005',
  '(2) Kerron CLEMENTSO',
  '',
  'Florida',
  '44.57📺 ',
  '3/12/2005'],
 ['(3) Randolph ROSSJRNorth Carolina A&T44.62 3/12/2022',
  '(3) Randolph ROSSJR',
  '',
  'North Carolina A&T',
  '44.62 ',
  '3/12/2022'],
 ['(4) Noah WILLIAMSJRLSU44.71 3/13/2021',
  '(4) Noah WILLIAMSJR',
  '',
  'LSU',
  '44.71 ',
  '3/13/2021'],
 ['(5) Elija GODWINSRGeorgia44.75📺 3/11/2023',
  '(5) Elija GODWINSR',
  '',
  'Georgia',
  '44.75📺 ',
  '3/11/2023'],
 ['\xa0\xa0\xa0\xa0— Godwin {2}SRGeorgia44.75 2/25/2023',
  '\xa0\xa0\xa0\xa0— Godwin {2}SR',
  '',
  'Georgia',
  '44.75 ',
  '2/25/2023'],
 ['(6) Kirani JAMESSOAlabama44.80📺 2/27/2011',
  '(6) Kirani JAMESSO',
  '',
  'Alabama',
  '44.80📺 ',
  '2/27/2011'],
 ['(7) Tyrell RICHARDSRSouth 

In [17]:
RealRealRows[1][4]

'44.57📺 '

In [18]:
len(RealRealRows)

39

In [19]:
OfficialTimes = []

In [20]:
for i in range(0,len(RealRealRows)):
    OfficialTimes.append(RealRealRows[i][4])
OfficialTimes

['44.52📺 ',
 '44.57📺 ',
 '44.62 ',
 '44.71 ',
 '44.75📺 ',
 '44.75 ',
 '44.80📺 ',
 '44.82 bb = Performance occurred on a 200m/220yd banked track ',
 '44.83 ',
 '44.85📺 ',
 '44.86 ',
 '44.93 ',
 '44.93  ',
 '44.99 ',
 '45.00 bb = Performance occurred on a 200m/220yd banked track ',
 '45.02 bb = Performance occurred on a 200m/220yd banked track ',
 '45.02 bb = Performance occurred on a 200m/220yd banked track ',
 '45.03 bb = Performance occurred on a 200m/220yd banked track ',
 '45.03 ',
 '45.03  ',
 '45.04 ',
 '45.04 bb = Performance occurred on a 200m/220yd banked track ',
 '45.05 ',
 '45.08 ',
 '45.09📺 ',
 '45.10 qq = Performance occurred during a non-final round. ',
 '45.10 ',
 '45.13 ',
 '45.14 ',
 '45.15 ',
 '45.15 qq = Performance occurred during a non-final round. ',
 '45.16 bb = Performance occurred on a 200m/220yd banked track ',
 '45.16 ',
 '45.19 bb = Performance occurred on a 200m/220yd banked track ',
 '45.21 ',
 '45.21 AA = Performance occurred at altitude (>1000 meters abo

In [21]:
RealRealRows[0][1]

'(1) Michael NORMANSO'

In [22]:
Names = []

for i in range(0,len(RealRealRows)):
    Names.append(RealRealRows[i][1])
Names

['(1) Michael NORMANSO',
 '(2) Kerron CLEMENTSO',
 '(3) Randolph ROSSJR',
 '(4) Noah WILLIAMSJR',
 '(5) Elija GODWINSR',
 '\xa0\xa0\xa0\xa0— Godwin {2}SR',
 '(6) Kirani JAMESSO',
 '(7) Tyrell RICHARDSR',
 '\xa0\xa0\xa0\xa0— Ross {2}JR',
 '(8) Fred KERLEYSR',
 '(9) Akeem BLOOMFIELDSO',
 '(10) Ryan WILLIEJR',
 '(10) LaShawn MERRITT',
 '\xa0\xa0\xa0\xa0— Ross {3}SO',
 '\xa0\xa0\xa0\xa0— Norman {2}SO',
 '\xa0\xa0\xa0\xa0— Bloomfield {2}SO',
 '\xa0\xa0\xa0\xa0— Kerley {2}SR',
 '(12) Kahmari MONTGOMERYSR',
 '(12) Deon LENDOREJR',
 '(12) Torrin LAWRENCESO',
 '(15) Champion ALLISONSR',
 '\xa0\xa0\xa0\xa0— Montgomery {2}SR',
 '(16) Jacory PATTERSONSR',
 '(17) Matthew MOORERSR',
 '(18) Christopher BAILEYSR',
 '\xa0\xa0\xa0\xa0— Kerley {3}SR',
 '\xa0\xa0\xa0\xa0— Lawrence {2}SO',
 '\xa0\xa0\xa0\xa0— Bailey {2}SR',
 '\xa0\xa0\xa0\xa0— Patterson {2}JR',
 '\xa0\xa0\xa0\xa0— Willie {2}JR',
 '\xa0\xa0\xa0\xa0— Lendore {2}SO',
 '(19) Wil LONDONSR',
 '(19) Mylik KERLEYSR',
 '\xa0\xa0\xa0\xa0— London {2}

In [23]:
OfficialReGexNames = []

In [24]:
for i in range(0,len(RealRealRows)):
    if Names[i].startswith("\xa0\xa0"):
        OfficialReGexNames.append(Names[i][4:])
    else:
        OfficialReGexNames.append(Names[i])
        
OfficialReGexNames

['(1) Michael NORMANSO',
 '(2) Kerron CLEMENTSO',
 '(3) Randolph ROSSJR',
 '(4) Noah WILLIAMSJR',
 '(5) Elija GODWINSR',
 '— Godwin {2}SR',
 '(6) Kirani JAMESSO',
 '(7) Tyrell RICHARDSR',
 '— Ross {2}JR',
 '(8) Fred KERLEYSR',
 '(9) Akeem BLOOMFIELDSO',
 '(10) Ryan WILLIEJR',
 '(10) LaShawn MERRITT',
 '— Ross {3}SO',
 '— Norman {2}SO',
 '— Bloomfield {2}SO',
 '— Kerley {2}SR',
 '(12) Kahmari MONTGOMERYSR',
 '(12) Deon LENDOREJR',
 '(12) Torrin LAWRENCESO',
 '(15) Champion ALLISONSR',
 '— Montgomery {2}SR',
 '(16) Jacory PATTERSONSR',
 '(17) Matthew MOORERSR',
 '(18) Christopher BAILEYSR',
 '— Kerley {3}SR',
 '— Lawrence {2}SO',
 '— Bailey {2}SR',
 '— Patterson {2}JR',
 '— Willie {2}JR',
 '— Lendore {2}SO',
 '(19) Wil LONDONSR',
 '(19) Mylik KERLEYSR',
 '— London {2}SR',
 '— Ross {4}SO',
 '— Lendore {3}',
 '(21) Tony MCQUAYSO',
 '(22) Bryce DEADMONSR',
 '— Bloomfield {3}SO']

In [25]:
CleanedNames = []

for i in range(0,len(RealRealRows)):
    if OfficialReGexNames[i].startswith("—"):
        CleanedNames.append(OfficialReGexNames[i][2:])
    if OfficialReGexNames[i].startswith("("):
        CleanedNames.append(OfficialReGexNames[i][4:])        
CleanedNames

['Michael NORMANSO',
 'Kerron CLEMENTSO',
 'Randolph ROSSJR',
 'Noah WILLIAMSJR',
 'Elija GODWINSR',
 'Godwin {2}SR',
 'Kirani JAMESSO',
 'Tyrell RICHARDSR',
 'Ross {2}JR',
 'Fred KERLEYSR',
 'Akeem BLOOMFIELDSO',
 ' Ryan WILLIEJR',
 ' LaShawn MERRITT',
 'Ross {3}SO',
 'Norman {2}SO',
 'Bloomfield {2}SO',
 'Kerley {2}SR',
 ' Kahmari MONTGOMERYSR',
 ' Deon LENDOREJR',
 ' Torrin LAWRENCESO',
 ' Champion ALLISONSR',
 'Montgomery {2}SR',
 ' Jacory PATTERSONSR',
 ' Matthew MOORERSR',
 ' Christopher BAILEYSR',
 'Kerley {3}SR',
 'Lawrence {2}SO',
 'Bailey {2}SR',
 'Patterson {2}JR',
 'Willie {2}JR',
 'Lendore {2}SO',
 ' Wil LONDONSR',
 ' Mylik KERLEYSR',
 'London {2}SR',
 'Ross {4}SO',
 'Lendore {3}',
 ' Tony MCQUAYSO',
 ' Bryce DEADMONSR',
 'Bloomfield {3}SO']

In [26]:
CleanedNames2 = []

for i in range(0,len(RealRealRows)):
    if CleanedNames[i].startswith(" "):
        CleanedNames2.append(CleanedNames[i][1:])
    else:
        CleanedNames2.append(CleanedNames[i])        
CleanedNames2

['Michael NORMANSO',
 'Kerron CLEMENTSO',
 'Randolph ROSSJR',
 'Noah WILLIAMSJR',
 'Elija GODWINSR',
 'Godwin {2}SR',
 'Kirani JAMESSO',
 'Tyrell RICHARDSR',
 'Ross {2}JR',
 'Fred KERLEYSR',
 'Akeem BLOOMFIELDSO',
 'Ryan WILLIEJR',
 'LaShawn MERRITT',
 'Ross {3}SO',
 'Norman {2}SO',
 'Bloomfield {2}SO',
 'Kerley {2}SR',
 'Kahmari MONTGOMERYSR',
 'Deon LENDOREJR',
 'Torrin LAWRENCESO',
 'Champion ALLISONSR',
 'Montgomery {2}SR',
 'Jacory PATTERSONSR',
 'Matthew MOORERSR',
 'Christopher BAILEYSR',
 'Kerley {3}SR',
 'Lawrence {2}SO',
 'Bailey {2}SR',
 'Patterson {2}JR',
 'Willie {2}JR',
 'Lendore {2}SO',
 'Wil LONDONSR',
 'Mylik KERLEYSR',
 'London {2}SR',
 'Ross {4}SO',
 'Lendore {3}',
 'Tony MCQUAYSO',
 'Bryce DEADMONSR',
 'Bloomfield {3}SO']

In [27]:
len(OfficialTimes)

39

In [28]:
len(OfficialReGexNames)

39

In [29]:
Schools = []

for i in range(0,len(OfficialTimes)):
    Schools.append(RealRealRows[i][3])
Schools

['Southern California',
 'Florida',
 'North Carolina A&T',
 'LSU',
 'Georgia',
 'Georgia',
 'Alabama',
 'South Carolina State',
 'North Carolina A&T',
 'Texas A&M',
 'Auburn',
 'Florida',
 'East Carolina',
 'North Carolina A&T',
 'Southern California',
 'Auburn',
 'Texas A&M',
 'Houston',
 'Texas A&M',
 'Georgia',
 'Florida',
 'Houston',
 'Florida',
 'Baylor',
 'Arkansas',
 'Texas A&M',
 'Georgia',
 'Arkansas',
 'Virginia Tech',
 'Florida',
 'Texas A&M',
 'Baylor',
 'Texas A&M',
 'Baylor',
 'North Carolina A&T',
 'Texas A&M',
 'Florida',
 'Texas A&M',
 'Auburn']

In [30]:
Times = []

In [31]:
for i in range(0,len(OfficialTimes)):
    Times.append(OfficialTimes[i][:5])
    
Times

['44.52',
 '44.57',
 '44.62',
 '44.71',
 '44.75',
 '44.75',
 '44.80',
 '44.82',
 '44.83',
 '44.85',
 '44.86',
 '44.93',
 '44.93',
 '44.99',
 '45.00',
 '45.02',
 '45.02',
 '45.03',
 '45.03',
 '45.03',
 '45.04',
 '45.04',
 '45.05',
 '45.08',
 '45.09',
 '45.10',
 '45.10',
 '45.13',
 '45.14',
 '45.15',
 '45.15',
 '45.16',
 '45.16',
 '45.19',
 '45.21',
 '45.21',
 '45.21',
 '45.22',
 '45.22']

In [32]:
IndoorMen400mDF = pd.DataFrame({'Names': CleanedNames2, 'Times': Times, 'Schools': Schools})

In [33]:
IndoorMen400mDF.to_csv('IndoorMens400m.csv', index=False)